# Getting Started with Mechanex

In [ ]:
import mechanex as mx

## 1. Loading a Model

Mechanex makes it easy to load models locally for experimentation. It automatically configures necessary hook points and SAE releases for you.

In [ ]:
# Load gpt2-small locally
mx.set_key("your-api-key")
mx.load("gpt2-small")

## 2. Advanced Sampling

You can generate text from the model with standard sampling methods like `top-k` and `top-p`.

In [ ]:
prompt = "The capital of France is"

print("Top-K Sampling:")
out_k = mx.generation.generate(prompt, max_tokens=10, sampling_method="top-k", top_k=50)
print(f"- {out_k}")

print("\nTop-P Sampling:")
out_p = mx.generation.generate(prompt, max_tokens=10, sampling_method="top-p", top_p=0.9)
print(f"- {out_p}")

## 3. Steering Vectors

Steering vectors allow you to nudge the model's internal representations toward specific behaviors. 

### Generate Vectors
We'll create a steering vector that encourages the model to be more "honest" using a few examples.

In [ ]:
honesty_vector_id = mx.steering.generate_vectors(
    prompts=["I tell the", "My statement is", "The truth is"],
    positive_answers=[" truth", " factual", " correct"],
    negative_answers=[" lie", " false", " wrong"],
    method="caa" # Contrastive Activation Addition
)
print(f"Generated steering vector with ID: {honesty_vector_id}")

### Save and Load Vectors

Mechanex allows you to export steering vectors to files so you can reuse them across sessions without recomputing.

In [ ]:
# Save the vector to disk
mx.steering.save_vectors(honesty_vector_id, "example_vector.json")

# Load it back as a dictionary of tensors
vectors = mx.steering.load_vectors("example_vector.json")

print(f"Loaded layers: {list(vectors.keys())}")

### Steer Generation

Now we apply the loaded vector during generation. Note how we can pass the vector dictionary directly or the ID.

In [ ]:
steered_output = mx.generation.generate(
    "Do I tell lies? Answer: ",
    max_tokens=20,
    steering_vector="3c676005-5610-4482-afa4-8b0405a228b0", # Passing the loaded dict
    steering_strength=2.0
)
print(f"Steered Output: {steered_output}")

## 4. Sparse Autoencoders (SAEs)

Mechanex integrates with `sae-lens` to automatically download and apply Sparse Autoencoders for concept steering. 

In [ ]:
behavior = mx.sae.create_behavior(
    "paris",
    prompts=["The city of light is", "I love visiting"],
    positive_answers=[" Paris", " Paris"],
    # Negative answers help isolate the concept
    negative_answers=[" London", " Rome"] 
)

### Steering with SAEs

You can also use these behaviors to steer the model towards specific SAE features.

In [ ]:
sae_steered = mx.sae.generate(
    "Do I want to go to rome or paris? Choose Paris or Rome. Answer:  ",
    behavior_names=["paris"],
    max_new_tokens=64,
    force_steering=["paris"]
)
print(f"SAE Steered: {sae_steered}")

## 5. Remote API Features


In [ ]:
mx.unload()

ads_output = mx.generation.generate(
    "Here is the story about the history of artificial intelligence:",
    sampling_method="ads",
    max_tokens=64
)
print(f"ADS Output: {ads_output}")

In [ ]:
# Create and save behaviors directly in your Axionic account for persistence
mx.sae.list_behaviors()